!['Relay_Problem'](Relay_Problem.png)

# Imports

In [1]:
from __future__ import print_function
from gurobipy import *

# Define Print Functions

In [2]:
def print_data():
    print('          ',end = '')
    for swimmer in swimmers:
        print('{:8}'.format(swimmer),end = '')
    print()
    for stroke in range(len(strokes)):
        print('{:10}'.format(strokes[stroke]),end = '')
        for swimmer in range(len(swimmers)):
            print (cost[(stroke,swimmer)],'   ',end = '')
        print()

In [3]:
def print_data_result():
    print('          ',end = '')
    for swimmer in swimmers:
        print('{:8}'.format(swimmer),end = '')
    stroke_before = None
    for v in m.getVars():
        
            stroke = int(v.VarName[1])
            swimmer = int(v.VarName[2])
            if stroke_before != stroke:
                print()
                print('{:8}'.format(strokes[stroke]),end = '')
            stroke_before = stroke
            if v.X > 0:
                print(' [',end = '')
            else:
                print('  ',end = '')
            print(cost[(stroke,swimmer)],end = '')
            if v.X > 0:
                print('] ',end = '')
            else:
                print('  ',end = '')        
    print('\n\nFor a total:',m.objVal,'seconds')

# Input the Data

!['Relay_Problem_Data'](Relay_Problem_Data.png)

In [4]:
strokes = ['Back','Breast','Fly','Free']
swimmers = ['Carl','Chris','David','Tony','Ken']
cost = {(0,0):37.7,(0,1):32.9,(0,2):33.8,(0,3):37.0,(0,4):35.4,
        (1,0):43.4,(1,1):33.1,(1,2):42.2,(1,3):34.7,(1,4):41.8,
        (2,0):33.3,(2,1):28.5,(2,2):38.9,(2,3):30.4,(2,4):33.6,
        (3,0):29.2,(3,1):26.4,(3,2):29.6,(3,3):28.5,(3,4):31.1}

# Review Data

In [5]:
print_data()

          Carl    Chris   David   Tony    Ken     
Back      37.7    32.9    33.8    37.0    35.4    
Breast    43.4    33.1    42.2    34.7    41.8    
Fly       33.3    28.5    38.9    30.4    33.6    
Free      29.2    26.4    29.6    28.5    31.1    


# Create Optimization Model

In [6]:
detail = False

m = Model('Relay')
m.setParam('OutputFlag', detail)

# Define the Variables 

### x(i,j) = { 1,  if stroke i is assigned to swimmer j;  0,  otherwise}
### where i is an element of strokes {0,1,2,3)
### and     j is an element of swimmers {0,1,2,3,4}

In [7]:
x = {}
for i in range(len(strokes)):
    for j in range(len(swimmers)):
        x[i,j] = m.addVar(obj=cost[(i,j)],vtype=GRB.BINARY, name='x%d%d' % (i,j))
m.update()

In [8]:
x

{(0, 0): <gurobi.Var x00>,
 (0, 1): <gurobi.Var x01>,
 (0, 2): <gurobi.Var x02>,
 (0, 3): <gurobi.Var x03>,
 (0, 4): <gurobi.Var x04>,
 (1, 0): <gurobi.Var x10>,
 (1, 1): <gurobi.Var x11>,
 (1, 2): <gurobi.Var x12>,
 (1, 3): <gurobi.Var x13>,
 (1, 4): <gurobi.Var x14>,
 (2, 0): <gurobi.Var x20>,
 (2, 1): <gurobi.Var x21>,
 (2, 2): <gurobi.Var x22>,
 (2, 3): <gurobi.Var x23>,
 (2, 4): <gurobi.Var x24>,
 (3, 0): <gurobi.Var x30>,
 (3, 1): <gurobi.Var x31>,
 (3, 2): <gurobi.Var x32>,
 (3, 3): <gurobi.Var x33>,
 (3, 4): <gurobi.Var x34>}

# Set up constraints

## One and Only One Swimmer Must be Assigned to Each Stroke

In [9]:
# For each strokes (i) one at a time, the sum of the swimmers(j) must be exactly one

for i in range(len(strokes)):
   m.addConstr(quicksum([x[i,j] for j in range(len(swimmers))]) == 1, 'stroke%d' % (i))

  ## A Swimmer Can Not Swim More Than Once

In [10]:
# For each swimmer (j) one at a time, the sum of strokes (i) must not be more than one

for j in range(len(swimmers)):
   m.addConstr(quicksum([x[i,j] for i in range(len(strokes))]) <= 1,  'swimmer%d' % (j))

# Call Optimize

In [11]:
m.optimize()

# Look at the LP (Linear Program)

In [12]:
 m.write('relay.lp')

# Get the Solution

In [13]:
m.objVal

126.2

In [14]:
m.getVars()

[<gurobi.Var x00 (value -0.0)>,
 <gurobi.Var x01 (value -0.0)>,
 <gurobi.Var x02 (value 1.0)>,
 <gurobi.Var x03 (value -0.0)>,
 <gurobi.Var x04 (value 0.0)>,
 <gurobi.Var x10 (value -0.0)>,
 <gurobi.Var x11 (value 0.0)>,
 <gurobi.Var x12 (value -0.0)>,
 <gurobi.Var x13 (value 1.0)>,
 <gurobi.Var x14 (value -0.0)>,
 <gurobi.Var x20 (value -0.0)>,
 <gurobi.Var x21 (value 1.0)>,
 <gurobi.Var x22 (value -0.0)>,
 <gurobi.Var x23 (value 0.0)>,
 <gurobi.Var x24 (value -0.0)>,
 <gurobi.Var x30 (value 1.0)>,
 <gurobi.Var x31 (value 0.0)>,
 <gurobi.Var x32 (value -0.0)>,
 <gurobi.Var x33 (value -0.0)>,
 <gurobi.Var x34 (value -0.0)>]

In [15]:
if m.status == GRB.status.OPTIMAL:
    for v in m.getVars():
        if v.X == 1:
            stroke = int(v.VarName[1])
            swimmer = int(v.VarName[2])
            print('{:8}'.format(strokes[stroke]+':'),
                  '{:8}'.format(swimmers[swimmer]),
                  cost[(stroke,swimmer)])
    print('{:16}'.format(''),'-----\n','{:15}'.format(''),m.objVal)
else:
    print('Bummer ',m.status)


Back:    David    33.8
Breast:  Tony     34.7
Fly:     Chris    28.5
Free:    Carl     29.2
                 -----
                 126.2


In [16]:
print_data_result()

          Carl    Chris   David   Tony    Ken     
Back      37.7    32.9   [33.8]   37.0    35.4  
Breast    43.4    33.1    42.2   [34.7]   41.8  
Fly       33.3   [28.5]   38.9    30.4    33.6  
Free     [29.2]   26.4    29.6    28.5    31.1  

For a total: 126.2 seconds
